## Setup

### Libraries

In [2]:
import numpy as np
import pandas as pd

import astropy

from astropy.timeseries import TimeSeries
from astropy.table import Table
from astropy import units as u
import plotly.graph_objects as go

import lightkurve as lk

import itertools 

from wotan import slide_clip
from wotan import flatten
from transitleastsquares import cleaned_array
from transitleastsquares import transitleastsquares

/Users/jacquelinemcshan/miniconda3/lib/python3.9/site-packages/lightkurve/config/__init__.py:119: UserWarning: The default Lightkurve cache directory, used by download(), etc., has been moved to /Users/jacquelinemcshan/.lightkurve/cache. Please move all the files in the legacy directory /Users/jacquelinemcshan/.lightkurve-cache to the new location and remove the legacy directory. Refer to https://docs.lightkurve.org/reference/config.html#default-cache-directory-migration for more information.
  warnings.warn(


## Data
For my senior experience project, I analysed  KIC 4840672, KIC 8162789, KIC 8631751, and KIC 11100170.  I am going to re-analyse these four candidates with better methods and then graph my results using plotly. The end goal is to put this up on my website. 


### Downloading the Data

In [3]:
search_result_1 = lk.search_lightcurve("KIC 4840672")
lc_collection_1 = search_result_1[search_result_1.author == "Kepler"].download_all()

### My Functions (makes things easier)

In [4]:
def lightcurve_data(data, window_length=0.5):

    SAP=[]
    PDCSAP=[]
    norm=[]
    time=[]

    for x in range(0, len(data)-1):
        sap_ind=data[x]['sap_flux'].to_value().tolist()
        pdcsap_ind=data[x]['flux'].to_value().tolist() # source of the flux in the table is PDCSAP FLUX
        norm_ind=data[x].normalize()['flux'].to_value().tolist()
        time_ind=data[x]['time'].to_value('bkjd').tolist()

        SAP.append(sap_ind)
        PDCSAP.append(pdcsap_ind)
        norm.append(norm_ind)
        time.append(time_ind)
    
    stitched=lc_collection.stitch()
    quality_index = np.where(stitched['quality'] == 0)[0]
    stitched=stitched[quality_index]
   
    stitched_flux=stitched['pdcsap_flux'].to_value().tolist() 
    stitched_time=stitched['time'].to_value('bkjd').tolist()


    clipped_flux = slide_clip(stitched_time, stitched_flux,
    window_length=window_length,
    low=3,
    high=2,
    method='mad',  # mad or std
    center='median'  # median or mean
    )
    
    flatten_lc=flatten( 
        stitched_time*u.day,                 # Array of time values
        clipped_flux,                 # Array of flux values
        window_length=window_length, method='biweight', return_trend=False)
    detrended_time, detrended_flux= cleaned_array(stitched_time, flatten_lc)

    return(PDCSAP, SAP, norm, time, detrended_flux, detrended_time, stitched) 


In [5]:
test1=lightcurve_data(lc_collection)

In [6]:
test1[4]

array([0.99998662, 1.0005899 , 1.00038307, ..., 1.00068831, 1.0001861 ,
       1.00034873])